In [1]:
import os
os.chdir('..')
os.getcwd()

'/home/leowyaoyang/fyp-graph-clustering'

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

import pickle 
import numpy as np
import pandas as pd
import torch
import os
from timeit import default_timer as timer

In [3]:
from core.GraphConvNet2 import GraphConvNet2
from core.DataEmbeddingGraph import DataEmbeddingGraph

In [4]:
if torch.cuda.is_available():
    print('cuda available')
    device = 'cuda'
else:
    print('cuda not available')
    device = 'cpu'

cuda available


In [5]:
parent_dir = os.path.abspath('..')
test_file = '/data/fasttext_test_tsne.pkl'
with open(parent_dir+test_file, 'rb') as f:
    [inputs, labels, X_emb] = pickle.load(f)

In [6]:
print(inputs.shape)
print(len(labels))
print(X_emb.shape)

(1000, 300)
1000
(1000, 3)


In [8]:
G = DataEmbeddingGraph(inputs, [])

In [9]:
net_parameters = {}
net_parameters['n_components'] = 3
net_parameters['D'] = 300 # input dimension
net_parameters['H'] = 50 # number of hidden units
net_parameters['L'] = 10 # number of hidden layers
net_parameters['n_channels'] = 1
net_parameters['n_units_1'] = net_parameters['n_units_2'] = net_parameters['H']

In [10]:
# Graph net
net = GraphConvNet2(net_parameters)
if torch.cuda.is_available(): net.cuda()
root = 'results/fasttext_preprocessed_tsne1/'
filename = root + 'graph_net5.pkl'
checkpoint = torch.load(filename, map_location=device)
net.load_state_dict(checkpoint['state_dict'])

In [11]:
# Net
if torch.cuda.is_available():   
    y_pred = net.forward(G).cpu().detach().numpy()
else:    
    y_pred = net.forward(G).detach().numpy()

In [12]:
n_plot = 500
X_plot = y_pred[:n_plot]
labels_plot = labels[:n_plot]

In [13]:
import bokeh.plotting as bp
from bokeh.plotting import save, show
from bokeh.models.glyphs import Text
from bokeh.models import HoverTool
from bokeh.io import output_notebook
from bokeh.transform import linear_cmap
from bokeh.palettes import Spectral6

output_notebook()

Loading BokehJS ...

In [14]:
title = 'fast text visualisation'

plot_fig = bp.figure(plot_width=800, plot_height=600,
                     title=title,
                     tools="pan, wheel_zoom, box_zoom, reset, hover, previewsave",
                     x_axis_type=None, y_axis_type=None, min_border=1)

# Use the third dimension as the color map
if X_plot.shape[1] == 3:
    colors = X_plot[:,2]
    mapper = linear_cmap(field_name='colors', palette=Spectral6 ,low=min(colors) ,high=max(colors))
    source = bp.ColumnDataSource(dict(x=X_plot[:,0], y=X_plot[:,1], text=labels_plot, colors=colors))
    plot_fig.text("x", "y", text="text", text_font_size='10px', text_color=mapper, source=source)
else:
    source = bp.ColumnDataSource(dict(x=X_plot[:,0], y=X_plot[:,1], text=labels_plot))
    plot_fig.text("x", "y", text="text", text_font_size='10px', source=source)

show(plot_fig)